__HTTP requests to the NASA Kennedy Space Center WWW server__

Fonte oficial do dateset: http://ita.ee.lbl.gov/html/contrib/NASA-HTTP.html

__Dados:__
- Jul 01 to Jul 31, ASCII format, 20.7 MB gzip compressed, 205.2 MB.
- Aug 04 to Aug 31, ASCII format, 21.8 MB gzip compressed, 167.8 MB.

__Sobre o dataset__: Esses dois conjuntos de dados possuem todas as requisições HTTP para o servidor da NASA Kennedy
Space Center WWW na Flórida para um período específico.

Os logs estão em arquivos ASCII com uma linha por requisição com as seguintes colunas:

- __Host fazendo a requisição__. Um hostname quando possível, caso contrário o endereço de internet se o nome não puder ser identificado.
- __Timestamp__ no formato [DIA/MÊS/ANO:HH:MM:SS YYYY] onde YYYY é o fuso horário. 
- __Requisição__
- __Código do retorno HTTP__
- __Total de bytes retornados__

    Fazendo importações e criando um acumulador personalizado para obter os hosts de linhas defeituosas:

In [2]:
from pyspark import AccumulatorParam

class StringAccumulatorParam(AccumulatorParam):
    def zero(self, initialValue=''):
        return ''
 
    def addInPlace(self, s1, s2):
        return s1.strip() + '|' + s2.strip()

acc_log = sc.accumulator('', StringAccumulatorParam())

    Criando DataFrame com as linhas que obedecem o padrão, e filtrando as linhas que não obedecem:

In [3]:
def filtro(linha, accumulator=acc_log):
    if len(linha.split(' ')) > 6:
        return True
    acc_log.add(linha)
    return False

def tratar(linha):
    host, _, _, timestamp, *resto = linha.split()
    byte_qtd = resto.pop(-1)
    return_code = resto.pop(-1)
    fuso = resto.pop(0)[:-1]
    req = ' '.join(resto)
    return host, timestamp[1:12], req, return_code, int(byte_qtd) if byte_qtd is not '-' else 0

    
_f = sc.textFile('*gz').filter(filtro).map(tratar)

df = _f.toDF().selectExpr('_1 as host', '_2 as timestamp', '_3 as req', '_4 as return_code', '_5 as byte_qtd')

    Executando uma ação para obter os resultados:

In [4]:
df.cache()
df.count()
df.show(10)

+--------------------+-----------+--------------------+-----------+--------+
|                host|  timestamp|                 req|return_code|byte_qtd|
+--------------------+-----------+--------------------+-----------+--------+
|   in24.inetnebr.com|01/Aug/1995|"GET /shuttle/mis...|        200|    1839|
|     uplherc.upl.com|01/Aug/1995|    "GET / HTTP/1.0"|        304|       0|
|     uplherc.upl.com|01/Aug/1995|"GET /images/kscl...|        304|       0|
|     uplherc.upl.com|01/Aug/1995|"GET /images/MOSA...|        304|       0|
|     uplherc.upl.com|01/Aug/1995|"GET /images/USA-...|        304|       0|
|ix-esc-ca2-07.ix....|01/Aug/1995|"GET /images/laun...|        200|    1713|
|     uplherc.upl.com|01/Aug/1995|"GET /images/WORL...|        304|       0|
|slppp6.intermind.net|01/Aug/1995|"GET /history/sky...|        200|    1687|
|piweba4y.prodigy.com|01/Aug/1995|"GET /images/laun...|        200|   11853|
|slppp6.intermind.net|01/Aug/1995|"GET /history/sky...|        200|    9202|

    Imprimindo linhas com defeito, para conferência:

In [5]:
linhas_defeituosas = acc_log.value.split('|')
for linha in linhas_defeituosas:
    if linha:
        print(linha)

alyssa.p


    Obtendo hosts unicos:

In [6]:
hosts_return = df.select('host', 'return_code')
hosts_unicos = hosts_return.select('host').distinct()


hosts_unicos.show(10)

+--------------------+
|                host|
+--------------------+
|ix-sea6-23.ix.net...|
|grimnet23.idirect...|
|      ird.scitex.com|
|      163.205.166.15|
|   chrism.tmx.com.au|
| boom.marblehead.com|
|        199.3.230.80|
|  enigma.idirect.com|
|ip26.abq-dialin.h...|
|   ppp20.coara.or.jp|
+--------------------+
only showing top 10 rows



                                        Número de hosts únicos:

In [7]:
hosts_unicos.count()

137978

------------------------------------------------------------------------

                                            Número de "404":

In [8]:
hosts_return404 = hosts_return.where('return_code = "404"')
hosts_return404.count()

20901

------------------------------------------------------------------------

                                    Os 5 URLs que mais causaram erro 404

In [9]:
url_404 = df.where('return_code = "404"').select('req')

from collections import Counter
contador = Counter()

# Poderia manter distribuído usando acumulador ou um map + reduce, como o usado mais adiante,
# Mas essa abordagem é mais simples, e a quantidade de itens não justifica o uso do spark
url_404 = url_404.collect()

for item in url_404:
    contador.update((item['req'].split()[1],))

print()
for url, qtd in contador.most_common(5):
    print(f'Quantidade:  {qtd} \t\t URL: {url}')


Quantidade:  2004 		 URL: /pub/winvn/readme.txt
Quantidade:  1732 		 URL: /pub/winvn/release.txt
Quantidade:  682 		 URL: /shuttle/missions/STS-69/mission-STS-69.html
Quantidade:  426 		 URL: /shuttle/missions/sts-68/ksc-upclose.gif
Quantidade:  384 		 URL: /history/apollo/a-001/a-001-patch-small.gif


------------------------------------------------------------------------

                                    Quantidade de erros 404 por dia.

In [10]:
datas = df.where('return_code = "404"').select('timestamp')

datas = datas.rdd.map(lambda l: l['timestamp'].split(':')[0])
unicas_por_dia = datas.map(lambda x:(x, 1)).reduceByKey(lambda x,y: x+y).cache()

unicas_por_dia_jl = unicas_por_dia.filter(lambda x: 'Jul' in x[0]).collect()
unicas_por_dia_ag = unicas_por_dia.filter(lambda x: 'Aug' in x[0]).collect()

unicas_por_dia_jl.sort()
unicas_por_dia_ag.sort()

unicas_por_dia_jl.extend(unicas_por_dia_ag)

print()
q_total=0
for dia, qtd in unicas_por_dia_jl:
    q_total += qtd
    print(f'Data: {dia}, \t Quantidade:{qtd}')

print(f'\nTotal de erros "404" retornados: {q_total}')


Data: 01/Jul/1995, 	 Quantidade:316
Data: 02/Jul/1995, 	 Quantidade:291
Data: 03/Jul/1995, 	 Quantidade:474
Data: 04/Jul/1995, 	 Quantidade:359
Data: 05/Jul/1995, 	 Quantidade:497
Data: 06/Jul/1995, 	 Quantidade:640
Data: 07/Jul/1995, 	 Quantidade:570
Data: 08/Jul/1995, 	 Quantidade:302
Data: 09/Jul/1995, 	 Quantidade:348
Data: 10/Jul/1995, 	 Quantidade:398
Data: 11/Jul/1995, 	 Quantidade:471
Data: 12/Jul/1995, 	 Quantidade:471
Data: 13/Jul/1995, 	 Quantidade:532
Data: 14/Jul/1995, 	 Quantidade:413
Data: 15/Jul/1995, 	 Quantidade:254
Data: 16/Jul/1995, 	 Quantidade:257
Data: 17/Jul/1995, 	 Quantidade:406
Data: 18/Jul/1995, 	 Quantidade:465
Data: 19/Jul/1995, 	 Quantidade:639
Data: 20/Jul/1995, 	 Quantidade:428
Data: 21/Jul/1995, 	 Quantidade:334
Data: 22/Jul/1995, 	 Quantidade:192
Data: 23/Jul/1995, 	 Quantidade:233
Data: 24/Jul/1995, 	 Quantidade:328
Data: 25/Jul/1995, 	 Quantidade:461
Data: 26/Jul/1995, 	 Quantidade:336
Data: 27/Jul/1995, 	 Quantidade:336
Data: 28/Jul/1995, 	 Quanti

Aparentemente não houveram acessos com erro dia 02 de Agosto e nos dias 29, 30 e 31 de Julho de 1995...

Por ser estranho procurei no arquivo por qualquer acesso nos mesmos dias.

Não ocorreram. Talvez o arquivo está incompleto ou o site não estivesse online.

------------------------------------------------------------------------

In [11]:
dados = df.select('timestamp', 'byte_qtd').rdd
dados = dados.map(lambda l: (l['timestamp'], l['byte_qtd']))

data_byte_qtd = dados.reduceByKey(lambda x,y: x+y)
# data_byte_qtd.take(10)

                                        Total de bytes retornados.

In [12]:
resultados = data_byte_qtd.sortByKey().collect()

r1 = sorted([r for r in resultados if 'Jul' in r[0]])
r2 = sorted([r for r in resultados if 'Aug' in r[0]])

r1.extend(r2)

print()


q_total = 0
for dia, qtd in r1:
    q_total += qtd
    print(f'Data: {dia}, \t Dados retornados: {qtd/1048576:.2f} MB')

print(f'\nTotal de dados retornados: {q_total/1073741824:.2f} GB')


Data: 01/Jul/1995, 	 Dados retornados: 1542.69 MB
Data: 02/Jul/1995, 	 Dados retornados: 1460.82 MB
Data: 03/Jul/1995, 	 Dados retornados: 1988.72 MB
Data: 04/Jul/1995, 	 Dados retornados: 1663.66 MB
Data: 05/Jul/1995, 	 Dados retornados: 1956.22 MB
Data: 06/Jul/1995, 	 Dados retornados: 2116.21 MB
Data: 07/Jul/1995, 	 Dados retornados: 1861.86 MB
Data: 08/Jul/1995, 	 Dados retornados: 885.15 MB
Data: 09/Jul/1995, 	 Dados retornados: 719.24 MB
Data: 10/Jul/1995, 	 Dados retornados: 1312.90 MB
Data: 11/Jul/1995, 	 Dados retornados: 1323.69 MB
Data: 12/Jul/1995, 	 Dados retornados: 1635.70 MB
Data: 13/Jul/1995, 	 Dados retornados: 2512.72 MB
Data: 14/Jul/1995, 	 Dados retornados: 1833.45 MB
Data: 15/Jul/1995, 	 Dados retornados: 930.86 MB
Data: 16/Jul/1995, 	 Dados retornados: 992.72 MB
Data: 17/Jul/1995, 	 Dados retornados: 1366.46 MB
Data: 18/Jul/1995, 	 Dados retornados: 1123.15 MB
Data: 19/Jul/1995, 	 Dados retornados: 1296.92 MB
Data: 20/Jul/1995, 	 Dados retornados: 1150.00 MB
Dat